In [ ]:
import sys
sys.path.append("..")

In [ ]:
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
from pprint import pprint

In [ ]:
NUM_ROWS = 1500000

In [ ]:
db = DuckDBDatabaseHelper("../meters.db")

In [ ]:
sql_query ="""

SELECT day,energy_sum, 
temperatureMin, 
temperatureMax,
uvIndex,
dewPoint,                  
Type,
date_part('month', day) as month
FROM  meters m LEFT JOIN info_household ih
ON m.LCLid = ih.LCLid
LEFT JOIN weather_daily_darksky_modified wd
ON m.day = wd.formatted_temperatureMaxTime_weather_daily_darksky
LEFT JOIN uk_bank_holidays uk_bh
ON m.day = uk_bh."Bank holidays"

WHERE date_part('year', day) = 2013

"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

In [ ]:
records[:5]

In [ ]:
COLS = [
"day","energy_sum", 
"temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type","month" ]

In [ ]:
df_model = pd.DataFrame(records,columns = COLS)

In [ ]:
df_model.head()

In [ ]:
df_model['Type_Binary'] = df_model['Type'].apply(lambda x: 1 if x is not None else 0)

In [ ]:
COLS_FEAT = ["temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type_Binary","month"]
COLS_LABEL = ["energy_sum"]

In [ ]:
df_model = df_model[COLS_FEAT + COLS_LABEL]

In [ ]:
df_model = df_model.dropna()
len(df_model)

In [ ]:
df_model.columns

In [ ]:
if len(df_model) < NUM_ROWS:
    NUM_ROWS2 = len(df_model)
else:
    NUM_ROWS2 = NUM_ROWS

In [ ]:
df_model_sample = df_model.sample(n = NUM_ROWS2)

In [ ]:
X = df_model_sample[COLS_FEAT]

In [ ]:
y = df_model_sample[COLS_LABEL]

In [ ]:
X.head()

In [ ]:
y

In [ ]:
df_model_sample.to_csv("../model_data/train.csv",index = False)